In [22]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.support.wait import WebDriverWait
from pyquery import PyQuery as pq
from urllib.parse import quote
import re
import time

chrome_options=webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
browser=webdriver.Chrome(chrome_options=chrome_options)
# browser=webdriver.Chrome()
wait=WebDriverWait(browser,10)
total=0

def index_main_page(id,urlspath):
    try:
        url='https://space.bilibili.com/'+str(id)+'/#/video'
        browser.get(url)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'#submit-video .list-list .list-item')))
        get_video_url(urlspath)
    except TimeoutException:
        index_main_page(id,urlspath)
        
def get_video_url(urlspath):
    html=browser.page_source
    doc=pq(html)
    items=doc('#submit-video .list-list .list-item').items()
    for item in items:
        with open (urlspath,'a',encoding='utf-8') as f:
            f.write('https:'+item.find('.cover').attr('href')+'\n')

            
def get_video_comments(urlspath,commentspath):
    f=open(urlspath)
    url=f.readline()
    count=1
    while url:
        print('正在爬取第 ',count,' 个视频。')
        parse_page(url,commentspath)
        url=f.readline()
#         测试结束删除break
        count+=1
    f.close()
    
def parse_page(url,commentspath):
    global total
    with open(commentspath,'a',encoding='utf-8') as f:
        f.write(url)
    browser.get(url)
    js='var q=document.documentElement.scrollTop=10000'
    browser.execute_script(js)
    time.sleep(0.1)
    browser.execute_script(js)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'.comment-list .list-item.reply-wrap')))
    html=browser.page_source
    doc=pq(html)
    total_pages=int(doc.find('.page-jump').children().text())
    print('total_pages:',total_pages)
    for page in range(1,total_pages+1):
        print('正在爬取第 ',page,' 页。')
        try:
            if page>1:
                submit=wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'.bottom-page.paging-box-big .next')))
                submit.click()
                browser.execute_script(js)
                time.sleep(0.1)
                wait.until(EC.text_to_be_present_in_element((By.CSS_SELECTOR,'.bottom-page.paging-box-big .current'),str(page)))
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'.comment-list .list-item.reply-wrap')))
                html=browser.page_source
                doc=pq(html)
        except StaleElementReferenceException as mag:
            print(msg)
            print('正在尝试重新获取...')
            browser.refresh()
            browser.execute_script(js)
            time.sleep(0.1)
            wait.until(EC.text_to_be_present_in_element((By.CSS_SELECTOR,'.bottom-page.paging-box-big .current'),str(page)))
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'.comment-list .list-item.reply-wrap')))
            html=browser.page_source
            doc=pq(html)
            
        print(doc('.bottom-page.paging-box-big .current').text() == str(page))
        items=doc('.comment-list .list-item.reply-wrap').items()
        for item in items:
            total+=1
            comment={}
            comment['id']=item.find('.name').attr('data-usercard-mid')
            comment['name']=item.find('.con .user .name').text().split(' ')[0]
            comment['content']=item.find('.text').text()
            
            comment['time']=re.search('(.*?:\d{2})',item('.time').text(),re.S).group(0)
            comment['like']=item('.like').text()
#             print(comment)
#             测试结束删除
            with open(commentspath,'a',encoding='utf-8') as f:
                for key in comment.keys():
                    f.write(key+':  '+comment[key].replace('\n','\t')+'    ')
                f.write('\n'*2)
            with open('/Users/limuyle/Desktop/webscrapy/comments_videos_for_wordcloud.txt','a',encoding='utf-8') as file:
                file.write(comment['content'])

        
                
                
            
    
    
    
        


def main(id,urlspath,commentspath):
    global browser
    index_main_page(id,urlspath)
    get_video_comments(urlspath,commentspath)
    browser.close()
            
if __name__ == '__main__':
    start_time=int(time.time())
    main(632887,'/Users/limuyle/Desktop/webscrapy/video_url.txt','/Users/limuyle/Desktop/webscrapy/video_comments.txt')
    end_time=int(time.time())
    print("程序运行了：",(end_time-start_time)/60,'分钟。')
    print('总评论数：',total)
            
        

正在爬取第  1  个视频。
total_pages: 10
正在爬取第  1  页。
True
正在爬取第  2  页。
True
正在爬取第  3  页。
True
正在爬取第  4  页。
True
正在爬取第  5  页。
True
正在爬取第  6  页。
True
正在爬取第  7  页。
True
正在爬取第  8  页。
True
正在爬取第  9  页。
True
正在爬取第  10  页。
True
正在爬取第  2  个视频。
total_pages: 14
正在爬取第  1  页。
True
正在爬取第  2  页。
True
正在爬取第  3  页。
True
正在爬取第  4  页。
True
正在爬取第  5  页。
True
正在爬取第  6  页。
True
正在爬取第  7  页。
True
正在爬取第  8  页。
True
正在爬取第  9  页。
True
正在爬取第  10  页。
True
正在爬取第  11  页。
True
正在爬取第  12  页。
True
正在爬取第  13  页。
True
正在爬取第  14  页。
True
正在爬取第  3  个视频。
total_pages: 12
正在爬取第  1  页。
True
正在爬取第  2  页。
True
正在爬取第  3  页。
True
正在爬取第  4  页。
True
正在爬取第  5  页。
True
正在爬取第  6  页。
True
正在爬取第  7  页。
True
正在爬取第  8  页。
True
正在爬取第  9  页。
True
正在爬取第  10  页。
True
正在爬取第  11  页。
True
正在爬取第  12  页。
True
正在爬取第  4  个视频。
total_pages: 13
正在爬取第  1  页。
True
正在爬取第  2  页。
True
正在爬取第  3  页。
True
正在爬取第  4  页。
True
正在爬取第  5  页。
True
正在爬取第  6  页。
True
正在爬取第  7  页。
True
正在爬取第  8  页。
True
正在爬取第  9  页。
True
正在爬取第  10  页。
True
正在爬取第  11  页。
True
正在爬取第  12  页。
True
